In [20]:
library(keras)
library(rlist)

data <- read.csv(file = 'C:/M4datasets/Hourly-train.csv', header = TRUE, sep = ',')
data <- data[, -1]

num_classes <- 1
batch_size <- 1
sample_size <- 1
sequence_length <-1
num_LSTM_layers <- 1
hidden_layer_size <- 100
win <- 5
epochs <- 1
num_units <- 5
train_size <- 0.7


train_test_split <- function(win, df1) {
    X_train <- df1[1: (nrow(df1) * train_size),-length(df1)]
    y_train <- df1[1: (nrow(df1) * train_size),length(df1), drop = F]
    X_test <- df1[(nrow(df1) * 0.7):nrow(df1),-length(df1)]
    y_test <- df1[(nrow(df1) * 0.7):nrow(df1),length(df1), drop = F]
    return(list(X_train, X_test, y_train, y_test))
}

gen_data <- function(win, row) {
    df <- data.frame(matrix(, ncol = win), stringsAsFactors=FALSE)
    colnames(df)<-  c(paste0("X", 1:(win -1)) , 'y')
    for (i in seq(1:as.integer(length(row) - win+1))) {
        df[i,] <- row[(i:(i + win - 1))]
    }
    return(df)
}


row_scale <- function(row) {
    return((row - mean(row))/sd(row))
}


layer_list <- function(num_LSTM_layers, hidden_layer_size) {
    layer_list <- list()
    for (i in (1 : (num_LSTM_layers))){
        if (i == (num_LSTM_layers)){
            layer_list <- list.append(layer_list,layer_lstm(units  = hidden_layer_size, 
                                                        input_shape      = c(win-1, sequence_length), 
                                                        batch_size       = batch_size,
                                                        return_sequences = F, 
                                                        stateful         = TRUE))
        } else {
            layer_list <- list.append(layer_list,layer_lstm(units  = hidden_layer_size, 
                                                        input_shape      = c(win-1, sequence_length), 
                                                        batch_size       = batch_size,
                                                        return_sequences = T, 
                                                        stateful         = TRUE))
        }
    }
    
    
    layer_list <- list.append(layer_list,   layer_dense(units = num_classes, 
                                            input_shape      = c(sequence_length, num_classes)))
    
    return(layer_list)
}


get_row <- function(data) {
    row <- data[c(1:sample_size),]
    row <- row[!is.na(row)]
    return(row)
}

row_decoder <- function(row, mean, stddev) {
    return((row*stddev) + mean)
}




unscaled_row <- get_row(data)
scaled_row <- row_scale(unscaled_row)
mean <- mean(unscaled_row)
stddev<- sd(unscaled_row)
df <- gen_data(win, scaled_row)


X_train <- data.frame(train_test_split(win, df)[1])
X_test <- data.frame(train_test_split(win, df)[2])
y_train <- data.frame(train_test_split(win, df)[3])
y_test <- data.frame(train_test_split(win, df)[4])

X_train <- array(as.matrix(X_train), dim = c(nrow(X_train), ncol(X_train), sequence_length))
y_train <- array(as.matrix(y_train), dim = c(nrow(y_train), ncol(y_train)))
X_test <- array(as.matrix(X_test), dim = c(nrow(X_test), ncol(X_test), sequence_length))
y_test <- array(as.matrix(y_test), dim = c(nrow(y_test), ncol(y_test)))


model <- keras_model_sequential(layers = layer_list(num_LSTM_layers, hidden_layer_size))

model %>% 
    compile(loss = 'mse', optimizer = 'adam')

for (i in 1:epochs) {
    model %>% fit(x          = X_train, 
                  y          = y_train, 
                  batch_size = batch_size,
                  epochs     = 20, 
                  verbose    = 2,
                  validation_split = 0.3,
                  shuffle    = FALSE)



    model %>% reset_states()
}

mse <- function(row1, row2){
    return(sum((row1 - row2)^2) / length(row1))
}

train_prediction <- model %>%
                    predict(X_train, batch_size = batch_size)

test_prediction <- model %>%
                    predict(X_test, batch_size = batch_size)


train_prediction <- row_decoder(train_prediction, mean, stddev)
test_prediction <- row_decoder(test_prediction, mean, stddev)
y_train <- row_decoder(y_train, mean, stddev)
y_test <- row_decoder(y_test, mean, stddev)


print(mse(train_prediction, y_train))
print(mse(test_prediction, y_test))


[1] 234.5212
[1] 252.7503
